In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load the datasets

In [ ]:
match_df = pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv")
ball_analysis_df = pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")

In [ ]:
match_df.head()

In [ ]:
ball_analysis_df.head()

**List out all teams**

In [ ]:
match_df["team1"].append(match_df["team2"]).unique()

Replace "Rising Pune Supergiant" to "Rising Pune Supergiants" 

In [ ]:
match_df.replace("Rising Pune Supergiant", value="Rising Pune Supergiants", inplace=True)

# Which is the team who won most of the matches?

In [ ]:
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt


In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(y=match_df["winner"], orient="h")

**Mumbai Indians won most**

# What decision make by teams after winning the toss?

In [ ]:
plt.figure(figsize=(12,8))
ax=sns.countplot(y="toss_winner", data=match_df, orient="h", hue="toss_decision")
plt.xlabel("Toss decision")

**Looks like more teams want to field first after winning the toss**

# How many teams are converting into match win after winning the toss?

In [ ]:
toss_winner = match_df["toss_winner"].value_counts()
winner_toss_df = match_df[match_df["toss_winner"]==match_df["winner"]]
winner_count_after_toss_win = winner_toss_df["winner"].value_counts()
both_win_dict = dict()
for items in winner_count_after_toss_win.iteritems():
    both_win_dict[items[0]] = float(items[1])/toss_winner[items[0]] * 100
plt.figure(figsize=(12,8))
sns.barplot(y=list(both_win_dict.keys()), x=list(both_win_dict.values()), orient="h")
plt.title("Match wins pecentage after toss win")
plt.xlabel("Percentage")

# Overall Win Percentage

In [ ]:
total_match = match_df["team1"].append(match_df["team2"])
total_match.value_counts()
denominator=total_match.value_counts()
b=match_df["winner"].value_counts()
percent={}
for items in b.iteritems(): 
    percent[items[0]]=float(items[1])/denominator[items[0]] * 100
ax=plt.figure(figsize=(12,8))
ax=sns.barplot(y=list(percent.keys()), x=list(percent.values()), orient="h")
ax.set_title("Pecentage Wins")
ax.set_xlabel("Percentage")
for rect in ax.patches:
    ax.text(rect.get_width(), rect.get_y() + rect.get_height() / 2,"%.1f%%"% rect.get_width() )

# Most Player of the match award

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(order=match_df['player_of_match'].value_counts()[:10].index, data=match_df, x="player_of_match")
# sns.scatterplot(x=match_df['player_of_match'].value_counts()[:10].index, y=match_df['player_of_match'].value_counts()[:10])
plt.title("Man of the Match Award")

# Which all team played the eliminator match?

In [ ]:
plt.figure(figsize=(12,8))
eliminator_df = match_df[match_df["eliminator"]=="Y"]
all_elim = eliminator_df["team1"].append(eliminator_df["team2"])
sns.countplot(y=eliminator_df["team1"].append(eliminator_df["team2"]), data=eliminator_df, orient="h")

Looks like Chennai Super Kings eliminator matches data is missing

# Let's do player analysis

# Which batsman dismissed more in 1st Over?

In [ ]:
first_over_wkt = ball_analysis_df[(ball_analysis_df["over"]==1) & (ball_analysis_df["is_wicket"]==1)]
plt.figure(figsize=(12,8))
sns.countplot(order=first_over_wkt['batsman'].value_counts()[:10].index, data=first_over_wkt, x="batsman")
plt.title("Batsman dismissed in First Over")

BB McCullum .. Shocking.. isn't it?

# Bowler who dismissed batsman in first over of innings

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(order=first_over_wkt['bowler'].value_counts()[:10].index, data=first_over_wkt, x="bowler")

Again Shocker :D

# Most Run by Batsman and Strike Rate (with minimum 100 balls faced)

In [ ]:
def runs_sr(batsman_grp):
    import operator
    fig, ax = plt.subplots(1,2, figsize=(15,8))

    score_map = batsman_grp["batsman_runs"].sum().sort_values(ascending=False)
    sns.barplot(ax=ax[0], y=list(score_map[0:10].index), x=list(score_map[0:10].values), orient="h")
    ax[0].set_title("Most Runs")

    ball_map = batsman_grp["ball"].count().sort_values(ascending=False)
    temp = {} 
    
    for item in ball_map.iteritems():
        # Played atleast these many balls
        if item[1] > 100:
            temp[item[0]] = float(score_map[item[0]])/item[1] * 100
    
    top_sr=dict(sorted(temp.items(), key=operator.itemgetter(1), reverse=True)[:10])
    sns.barplot(ax=ax[1], y=list(top_sr.keys()), x=list(top_sr.values()), orient="h")
    ax[1].set_title("Strike Rate")

In [ ]:
batsman_grp = ball_analysis_df.groupby(["batsman"])
runs_sr(batsman_grp)

# Batsman who scored most in powerplay (0-6)

In [ ]:
first_six_overs=ball_analysis_df[ball_analysis_df["over"].isin([1,2,3,4,5,6])]
batsman_grp = first_six_overs.groupby(["batsman"])
runs_sr(batsman_grp)

# Batsman score most in last 5 overs and Batsman with good strike rate

In [ ]:
last_overs=ball_analysis_df[ball_analysis_df["over"].isin([15,16,17,18,19])]
batsman_grp = last_overs.groupby(["batsman"])
runs_sr(batsman_grp)

# Lets talk about bowlers

In [ ]:
def wkt_n_economy(bowler_grp):
    import operator
    fig, ax = plt.subplots(1,2, figsize=(15,8))

    wkt_map = bowler_grp["is_wicket"].sum().sort_values(ascending=False)
    sns.barplot(ax=ax[0], y=list(wkt_map[0:10].index), x=list(wkt_map[0:10].values), orient="h")
    ax[0].set_title("Most Wicket")
    
    runs_count = bowler_grp["total_runs"].sum().sort_values(ascending=False)
    ball_count = bowler_grp["ball"].count().sort_values(ascending=False)
    extra_count = bowler_grp["extra_runs"].count().sort_values(ascending=False)
    economy_rate = dict()
    for item in ball_count.iteritems():
        if item[1] > 100:
            economy_rate[item[0]] = float(runs_count[item[0]])/ ((ball_count[item[0]])/6)
    top_er=dict(sorted(economy_rate.items(), key=operator.itemgetter(1))[:10])
    sns.barplot(ax=ax[1], y=list(top_er.keys()), x=list(top_er.values()), orient="h")
    ax[1].set_title("Most Economy")


# All time highest wicket taker and best economy

In [ ]:
bowler_grp = ball_analysis_df.groupby(["bowler"])
wkt_n_economy(bowler_grp)

# Most wicket and less economy in first 6 overs

In [ ]:
bowler_grp = first_six_overs.groupby(["bowler"])
wkt_n_economy(bowler_grp)

# Most Wickets and economy in last 5 overs

In [ ]:
bowler_grp = last_overs.groupby(["bowler"])
wkt_n_economy(bowler_grp)

# Lets analyze which both batsmen and bowler together (which batsman get out on which bowler)

In [ ]:
def batsman_wicket_analysis(name):
    batsman_data = ball_analysis_df[ball_analysis_df["batsman"]==name]
    wkt_data = batsman_data[batsman_data["is_wicket"]==1]
    bowler = wkt_data["bowler"].value_counts()
    dissmisal_type = wkt_data["dismissal_kind"].value_counts()
    
    fig, ax = plt.subplots(1,3, figsize=(16,8))
    
    best_bowler = wkt_data['bowler'].value_counts()[:10].index
    sns.countplot(ax=ax[0],order=wkt_data['bowler'].value_counts()[:10].index, data=wkt_data, y="bowler")
    ax[0].set_title("Bowler")
    
    best_bowler_df=wkt_data[wkt_data['bowler']==best_bowler[0]]
    sns.countplot(ax=ax[1],order=best_bowler_df['dismissal_kind'].value_counts()[:10].index, data=best_bowler_df, y="dismissal_kind")
    ax[1].set_title(str(best_bowler[0])+"- Dismissal Type")
    for rect in ax[1].patches:
        ax[1].text(rect.get_width(), rect.get_y() + rect.get_height() / 2, rect.get_width() )
    
    ax[2].pie(list(wkt_data['dismissal_kind'].value_counts()[:10]), labels=list(wkt_data['dismissal_kind'].value_counts()[:10].index),autopct='%1.1f%%',
        shadow=True,)
    ax[2].set_title("Overall Dismissal")

In [ ]:
batsman_wicket_analysis("V Kohli")

In [ ]:
batsman_wicket_analysis("AD Russell")

In [ ]:
match_df['year'] = match_df.apply(lambda row: row.date.split("-")[0], axis=1)

In [ ]:
merged_year = pd.merge(ball_analysis_df, match_df, on='id', how='left')
merged_year.head()

In [ ]:
unique_years = merged_year["year"].unique()
unique_years

# Batsman's Performance Comparision among the different IPL season

In [ ]:
def player_performance(name):
    runs_map = dict()
    name_df = merged_year[merged_year["batsman"]==name]
    for year in unique_years:
        temp_df = name_df[name_df["year"]==year]
        runs_map[year]=temp_df["batsman_runs"].sum()
    return runs_map

In [ ]:
plt.figure(figsize=(12,8))

v_runs = player_performance("V Kohli")
sns.lineplot(x=list(v_runs.keys()), y=list(v_runs.values()), label="V Kohli")

r_runs = player_performance("RG Sharma")
sns.lineplot(x=list(r_runs.keys()), y=list(r_runs.values()), label="RG Sharma")

plt.xlabel("Players")
plt.ylabel("Runs in an IPL")
plt.legend()

# Finding out the Orange and Purple caps

In [ ]:
i = 0
orange_cap_df = pd.DataFrame(columns=['year', 'name', 'runs'])
purple_cap_df = pd.DataFrame(columns=['year', 'name', 'wickets'])
for year in unique_years:
    year_df = merged_year[merged_year["year"]==year]

    batsman_grp = year_df.groupby(["batsman"])
    score_map = batsman_grp["batsman_runs"].sum().sort_values(ascending=False)
    orange_cap_df.loc[i] = year, score_map.index[0], score_map.values[0]

    bowler_grp = year_df.groupby(["bowler"])
    wkt_map = bowler_grp["is_wicket"].sum().sort_values(ascending=False)
    purple_cap_df.loc[i] = year, wkt_map.index[0], wkt_map.values[0]
    
    i+=1


# Orange Cap Holders

In [ ]:
plt.figure(figsize=(12,8))
ax=sns.scatterplot(x="year", y="runs", data=orange_cap_df)
for i, data in orange_cap_df.iterrows():
    ax.text(data['year'], data['runs']+10, str(data['name']))
plt.title("Most runs in a IPL season")
plt.xlabel("Players")
plt.ylabel("Runs")

# Purple Cap Holders

In [ ]:
plt.figure(figsize=(12,8))
ax=sns.scatterplot(x="year", y="wickets", data=purple_cap_df)
for i, data in purple_cap_df.iterrows():
    ax.text(data['year'], data['wickets']+0.2, str(data['name']))
plt.title("Most Wickets in a IPL season")
plt.xlabel("Players")
plt.ylabel("Wickets")

If you like analysis, then upvote this :)